In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [6]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = np.array(json.loads(open(data_path, "r").read())['col_name'])

create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])

dummy_theory_output = torch.tensor(mode_within(dummy_data[dummy_data>20],1))
theory_output = torch.tensor(mode_within(data[data>20],1))

In [ ]:
scales = [2]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        filter = (X>20)
        min_X = torch.min(X)
        fil_X = torch.where(filter, X, min_X-1)
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)


verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_10339/1524068298.py:14: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_10339/1524068298.py:14: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_e

In [8]:
# prover calculates settings, send to verifier
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        filter = (X>20)
        min_X = torch.min(X)
        fil_X = torch.where(filter, X, min_X-1)
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)

prover_gen_settings(data_path,selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_10339/2893268206.py:15: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])


Theory_output:  tensor(40., dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":299,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_10339/2893268206.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.logical_or(torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal, torch.abs(min_X-1-ele[0])<=torch.abs(0.01*ele[0])) for ele in X[0]])


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.478428840637207 seconds
Theory output:  tensor(40., dtype=torch.float64)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 40.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12341676197686541490, 2627393525778350065, 16625494184434727973, 1478518078215075360]]], 'proof': '252867d19f4ff0baedacc9c5e2100559f97573bd69269664bbaa26cfc97bea760632b9f70c48675d8530e3856c32ead9a3dea08d4a8242c64b76b983a08fad222205cfdf5c8b026e027e6af2d85c4d65152e5b4b9c25cda8c706a11cb4dc876d03fa19841d5fd69f9d6f1d9b8fd6f0b9c899ecbc79c4532984f29fec51c8f2db04964d882ffc93dca7402a04399c1d9cb500a960d609303f994a7ff3776dcb9028ec6a417bf87def071aa68798a76c2232fc9a3a573fc9150d4645ca702347db2d2ef8a0cd515ba700a6820f1ff28ed7a61495983f82e4a3cefc88056437e84929aac812dceec0c7f7b69d5c95391083dcd630c12a6aff42734df06052f8dc7027433e06d01128d0329dd438f35d46d93c8d52fdc319dbc75094ad791584a2aa2aa7622a8083ad6e497c3e9a157598452b8153051ac579ee489a727348236538143fe33beca164a73c4ce08c939eccc2f2c6f523b10e18d09c620eb551d669

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

num_inputs:  1
prf instances:  [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12341676197686541490, 2627393525778350065, 16625494184434727973, 1478518078215075360]]]
proof boolean:  1.0
proof result 1 : 40.0
verified
